# Pyterrier - Example Experiment

# Preparation

In [1]:
!pip install python-terrier

  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-ootrg_ul/python-terrier
  Running command git clone -q https://github.com/terrier-org/pyterrier.git /tmp/pip-install-ootrg_ul/python-terrier
     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.1.3-cp36-none-any.whl size=29595 sha256=8ce120bd9f33342a9335a6a50c62a7af372b56e009f689b49daceb1076cda604
  Stored in directory: /tmp/pip-ephem-wheel-cache-ge77gm06/wheels/cc/bb/69/836d846a92c787b35ca6478119c0033762ab2b95d866eeb288
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=42f53397a620947fedd9e3dbe6c7fff427cde4b293d81f00ca986bff2e98ccd3
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
  Created wheel for pytrec-eval: filename=pytrec_eval-0.4-cp36-cp36m-linux_x86_64.whl size=273829 sha256=6693a7d374a09011262da29b1f

# Init 

You must run `pt.init()` before other pyterrier functions and classes

Arguments:
 - `version` - Terrier platform version e.g. `"5.2"`    
 - `mem` - megabytes allocated to Java e.g. `4096`      


In [1]:
import pyterrier as pt
if not pt.started():
  pt.init()

/Users/craigmacdonald/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
PyTerrier 0.4.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


Again, we're using the Dataset interface to quickly access a test collection.

In [2]:
vaswani = pt.datasets.get_dataset("vaswani")

# Experiment

Тhe `experiment` function allows you to perform retrieval and evaluation in a declarative fashoin, allowing a simple function call with multiple retrieval systems to be executed using a single function call.

First create the BatchRetrieve objects with the configurartion that you wish to use

In [3]:
TF_IDF = pt.BatchRetrieve(vaswani.get_index(), wmodel="TF_IDF")
BM25 = pt.BatchRetrieve(vaswani.get_index(), wmodel="BM25")
PL2 = pt.BatchRetrieve(vaswani.get_index(), wmodel="PL2")

Call `pt.Experiment` with the list of retrieval objects, topics, qrels and list of metrics

Optional arguments:    
 - `perquery=False` - Show the results for each query instad of the mean    
 - `dataframe=True` - Return the result as a dataframe if True, or as a dictionary if False
 - `round=4` - round all measures to 4 decimal places
 

In [4]:
pt.Experiment(
    [TF_IDF,BM25,PL2],
    vaswani.get_topics(),
    vaswani.get_qrels(),
    ['map','ndcg'])

,name,map,ndcg
0,BR(TF_IDF),0.290905,0.615367
1,BR(BM25),0.296517,0.621197
2,BR(PL2),0.276264,0.601225


## Declaring a baseline

Use the `baseline=` kwarg to specify which system should be considered the baseline in this experiment. 
This will enable significance testing (paired t-test), as well as showing the number of queries improved and degraded 
wrt. the baseline.



In [5]:
pt.Experiment(
    [TF_IDF,BM25,PL2],
    vaswani.get_topics(),
    vaswani.get_qrels(),
    ['map','ndcg'],
    baseline=0)

,name,map,ndcg,map +,map -,map p-value,ndcg +,ndcg -,ndcg p-value
0,BR(TF_IDF),0.290905,0.615367,NaN,NaN,NaN,NaN,NaN,NaN
1,BR(BM25),0.296517,0.621197,46.0,45.0,0.237317,45.0,46.0,0.143493
2,BR(PL2),0.276264,0.601225,13.0,77.0,0.008827,17.0,73.0,0.002583


We can also apply correcion for multiple testing. Use the `correction=` kwarg - 'b' is a short hand for 'bonferroni'. The additional columns
"map p-value corrected" "map reject" are added. The former shows what the new, adjusted, p-value is, and the latter shows if the 
null hypothesis can rejected at $\alpha=0.05$.

In [6]:
pt.Experiment(
    [TF_IDF,BM25,PL2],
    vaswani.get_topics(),
    vaswani.get_qrels(),
    ['map','ndcg'],
    baseline=0,
    correction='b')

,name,map,ndcg,map +,map -,map p-value,map reject,map p-value corrected,ndcg +,ndcg -,ndcg p-value,ndcg reject,ndcg p-value corrected
0,BR(TF_IDF),0.290905,0.615367,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,NaN
1,BR(BM25),0.296517,0.621197,46.0,45.0,0.237317,False,0.711951,45.0,46.0,0.143493,False,0.43048
2,BR(PL2),0.276264,0.601225,13.0,77.0,0.008827,True,0.026482,17.0,73.0,0.002583,True,0.00775
